In [100]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import torch
from sklearn.metrics import confusion_matrix, mean_squared_error
import os
import pickle
import time
from itertools import chain, combinations
from collections import Counter
from Graphs import matrix_to_graph, graph_to_matrix, ErdosRenyiGraph, dRegularGraph
from ShortestDistanceAlgorithms import shortestDistance_allNodes_networkx, shortestDistance_allNodes_Bourgain, shortestDistance_allNodes_Sarma, onlineShortestPath_Bourgain, onlineShortestPath_Sarma
from Models import build

In [65]:
G,_,_ = dRegularGraph(100,3)
shortest_path_lengths = nx.shortest_path_length(G,sources=[5,6])
print(shortest_path_lengths)



TypeError: shortest_path_length() got an unexpected keyword argument 'sources'

In [102]:
G,_,_ = dRegularGraph(100,2)
dist = np.zeros((len(G.nodes()),len(G.nodes())))
shortest_path_lengths = nx.shortest_path_length(G,source=None,target=None)
for source_node, lengths in shortest_path_lengths:
    keys_list = [n for n in range(len(G.nodes())) if n not in lengths.keys()]
    values_list = np.ones(len(keys_list))*np.inf
    new_dict = dict(zip(keys_list, values_list))
    lengths.update(new_dict)
    lengths = dict(sorted(lengths.items(), key=lambda item: item[0]))
    dist[int(source_node)] = list(lengths.values())
print(dist)
    
    


[[ 0. 36.  7. ...  2. 29. 33.]
 [36.  0. 42. ... 38.  7. 16.]
 [ 7. 42.  0. ...  5. 36. 26.]
 ...
 [ 2. 38.  5. ...  0. 31. 31.]
 [29.  7. 36. ... 31.  0. 23.]
 [33. 16. 26. ... 31. 23.  0.]]


In [118]:
G,_,_ = dRegularGraph(100,2)
shortest_path_lengths = nx.shortest_path_length(G,source=5)
keys_list = [n for n in range(len(G.nodes())) if n not in shortest_path_lengths.keys()]
values_list = np.ones(len(keys_list))*np.inf
new_dict = dict(zip(keys_list, values_list))
lengths.update(new_dict)
lengths = dict(sorted(lengths.items(), key=lambda item: item[0]))
dist = np.array(list(lengths.values()))
print(dist)

[10. 26. inf 13. inf 12. 19. 11. inf 14. inf  8. 34. 38. inf inf  6. 21.
 17. 15. inf inf inf 15. inf 16. inf inf inf inf inf  4. inf inf  9. inf
 inf inf  1. 32. inf inf inf inf  2. 31. inf inf 23. inf inf inf 41.  3.
 30. 11. inf 29. inf inf 25. 32. inf 16. inf  8. inf inf 28. inf 24. 37.
 13. inf  5. inf inf inf 39. 38. 34.  2. inf 20. inf 25. inf inf 40. inf
 inf inf 40. 21. inf inf 31. 12. 19. inf]


In [121]:
def shortestDistances(G,source=None,target=None):
    num_nodes = len(G.nodes())
    if source == None and target == None:
        dist = np.zeros((len(G.nodes()),len(G.nodes())))
        if 'weight' in G.graph:
            shortest_path_lengths = nx.shortest_path_length(G,weight="weight")
        else:
            shortest_path_lengths = nx.shortest_path_length(G)
        for source_node, lengths in shortest_path_lengths:
            keys_list = [n for n in range(num_nodes) if n not in lengths.keys()]
            values_list = np.ones(len(keys_list))*np.inf
            lengths.update(dict(zip(keys_list, values_list)))
            lengths = dict(sorted(lengths.items(), key=lambda item: item[0]))
            dist[int(source_node)] = list(lengths.values())
    elif target == None:
        if 'weight' in G.graph:
            shortest_path_lengths = nx.shortest_path_length(G,source=source,weight="weight")
        else:
            shortest_path_lengths = nx.shortest_path_length(G,source=source)
        keys_list = [n for n in range(len(G.nodes())) if n not in shortest_path_lengths.keys()]
        values_list = np.ones(len(keys_list))*np.inf
        shortest_path_lengths.update(dict(zip(keys_list, values_list)))
        shortest_path_lengths = dict(sorted(shortest_path_lengths.items(), key=lambda item: item[0]))
        dist = np.array(list(shortest_path_lengths.values()))
    elif source == None:
        if 'weight' in G.graph:
            shortest_path_lengths = nx.shortest_path_length(G,target=target,weight="weight")
        else:
            shortest_path_lengths = nx.shortest_path_length(G,target=target)
        keys_list = [n for n in range(len(G.nodes())) if n not in shortest_path_lengths.keys()]
        values_list = np.ones(len(keys_list))*np.inf
        shortest_path_lengths.update(dict(zip(keys_list, values_list)))
        shortest_path_lengths = dict(sorted(shortest_path_lengths.items(), key=lambda item: item[0]))
        dist = np.array(list(shortest_path_lengths.values()))
    else:
        try:
            if 'weight' in G.graph:
                dist = nx.shortest_path_length(G,source=source,target=target,weight="weight")
            else:
                dist = nx.shortest_path_length(G,source=source,target=target)
        except:
            dist = np.inf
    return dist

In [145]:
directed_graph = nx.DiGraph()
directed_graph.add_edge(1, 2, weight=4)
directed_graph.add_edge(2, 1, weight=5)
directed_graph.add_edge(2, 3, weight=5)
directed_graph.add_edge(3, 1, weight=2)

# Convert the directed graph to an undirected graph (preserving weights)
undirected_graph = directed_graph.to_undirected()

# Print edges and weights in the undirected graph
for edge in undirected_graph.edges(data=True):
    print(f"Edge: {edge[0]} - {edge[1]}, Weight: {edge[2]['weight']}")

Edge: 1 - 2, Weight: 5
Edge: 1 - 3, Weight: 2
Edge: 2 - 3, Weight: 5


In [149]:
import networkx as nx

# Create a graph with weighted edges
G = nx.Graph()
G.add_edge(1, 2, weight=3)
G.add_edge(2, 3, weight=5)
G.add_edge(3, 1, weight=2)

# Choose a node
node_to_check = 2

# Get neighbors of the node
neighbors = list(G.neighbors(node_to_check))

# Get edge weights between the chosen node and its neighbors
edge_weights = {neighbor: G.get_edge_data(node_to_check, neighbor)['weight'] for neighbor in neighbors}

print(f"Neighbors of node {node_to_check}: {neighbors}")
print(f"Edge weights from node {node_to_check}: {edge_weights}")
for n in G.neighbors(node_to_check):
    print(n)



Neighbors of node 2: [1, 3]
Edge weights from node 2: {1: 3, 3: 5}
1
3


In [136]:
G,_,_ = dRegularGraph(100,2)
shortestDistances(G,source=None,target=4)

array([inf, inf, inf, inf,  0., inf, inf, inf, inf, inf, inf,  2., inf,
       inf,  3., inf, inf, inf, inf, inf,  4., inf, inf, inf, inf, inf,
       inf, inf, inf,  6.,  5., inf,  1., inf, inf, inf, inf, inf, inf,
       inf,  1., inf, inf, inf, inf, inf, inf,  3., inf, inf, inf, inf,
       inf, inf, inf,  4., inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf,  2., inf, inf, inf,  5., inf])

In [107]:
G,_,_ = dRegularGraph(100,2)
shortest_path_lengths = nx.shortest_path_length(G,target=5)
keys_list = [n for n in range(len(G.nodes())) if n not in lengths.keys()]
values_list = np.ones(len(keys_list))*np.inf
new_dict = dict(zip(keys_list, values_list))
lengths.update(new_dict)
lengths = dict(sorted(lengths.items(), key=lambda item: item[0]))
dist = np.array(list(lengths.values()))
print(dist)

[10. 26. 17. 13. inf 12. 19. 11. 24. 14. 41.  8. 34. 38. 22. 30.  6. 21.
 17. 15. 14.  4.  5. 15. 18. 16. inf inf 29. 27. 35.  4. 39. inf  9. 33.
 inf 37.  1. 32. 36. 42. inf  3.  2. 31. 35. 20. 23.  1.  7.  9. 41.  3.
 30. 11. inf 29. 36. inf 25. 32. inf 16. 26.  8. 23.  0. 28. 18. 24. 37.
 13. 22.  5. 28. 10. inf 39. 38. 34.  2. inf 20. inf 25. 33. 27. 40. inf
  7. inf 40. 21. inf  6. 31. 12. 19. 42.]


In [58]:
shortest_path_lengths[5]


TypeError: 'generator' object is not subscriptable

In [ ]:
import networkx as nx

# Creating a simple directed graph
G = nx.DiGraph()
G.add_edges_from([(1, 2), (1, 3), (2, 4), (3, 4), (4, 5)])

# Specifying source nodes
source_nodes = [1, 3]

# Calculating the shortest path lengths from specified source nodes to all other nodes
shortest_path_lengths = nx.shortest_path_length(G)

# Displaying the results
for source_node, lengths in shortest_path_lengths:
    print(source_node,lengths)
    #for target_node, length in lengths.items():
        #print(f"Shortest path length from Node {source_node} to Node {target_node}: {length}")

NodeNotFound: Source [1, 3] is not in G

In [ ]:
G.nodes()

{}

In [ ]:
for source_node, lengths in shortest_path_lengths:
    for target_node, length in lengths.items():
        print(f"Shortest path length from Node {source_node} to Node {target_node}: {length}")

Shortest path length from Node 23 to Node 23: 0
Shortest path length from Node 23 to Node 68: 1
Shortest path length from Node 23 to Node 54: 1
Shortest path length from Node 23 to Node 64: 1
Shortest path length from Node 23 to Node 52: 2
Shortest path length from Node 23 to Node 33: 2
Shortest path length from Node 23 to Node 57: 2
Shortest path length from Node 23 to Node 93: 2
Shortest path length from Node 23 to Node 78: 2
Shortest path length from Node 23 to Node 26: 2
Shortest path length from Node 23 to Node 29: 3
Shortest path length from Node 23 to Node 27: 3
Shortest path length from Node 23 to Node 83: 3
Shortest path length from Node 23 to Node 92: 3
Shortest path length from Node 23 to Node 44: 3
Shortest path length from Node 23 to Node 50: 3
Shortest path length from Node 23 to Node 6: 3
Shortest path length from Node 23 to Node 87: 3
Shortest path length from Node 23 to Node 16: 3
Shortest path length from Node 23 to Node 4: 3
Shortest path length from Node 23 to Node 